## Qubits Cannot be Used after Deallocation

Failed Example

In [1]:
from guppylang import guppy
from guppylang.std.builtins import owned
from guppylang.std.quantum import qubit, h, cx, measure

@guppy
def owned_qubits(
   alice: qubit @ owned,
   bob: qubit
) -> bool:
    h(alice)
    cx(alice, bob)
    alice_c: bool = measure(alice)
    h(alice) # invalid operation
    return alice_c

owned_qubits.check()

Error: Copy violation (at <In[1]>:13:6)
   | 
11 |     cx(alice, bob)
12 |     alice_c: bool = measure(alice)
13 |     h(alice) # invalid operation
   |       ^^^^^ Variable `alice` with non-copyable type `qubit` cannot be
   |             borrowed ...
   | 
12 |     alice_c: bool = measure(alice)
   |                             ----- since it was already consumed here

Guppy compilation failed due to 1 previous error


Successful Example

In [3]:
@guppy
def owned_qubits(
   alice: qubit @ owned,
   bob: qubit
) -> bool:
    h(alice)
    cx(alice, bob)
    alice_c: bool = measure(alice)
    alice1 = qubit()
    h(alice1) # invalid operation
    alice1_c = measure(alice1)
    return alice_c

owned_qubits.check()

## A Qubit cannot be used more than once

Failed Example

In [8]:
from guppylang import guppy
from guppylang.std.builtins import owned
from guppylang.std.quantum import qubit, h, cx, measure

@guppy
def borrow_qubits(
    alice: qubit,
    bob: qubit
) -> None:
    x(alice)
    h(alice)
    cx(alice, alice) # invalid operation 

borrow_qubits.check()

Error: Copy violation (at <In[8]>:12:14)
   | 
10 |     x(alice)
11 |     h(alice)
12 |     cx(alice, alice) # invalid operation
   |               ^^^^^ Variable `alice` with non-copyable type `qubit` cannot be
   |                     borrowed ...
   | 
12 |     cx(alice, alice) # invalid operation
   |        ----- since it was already borrowed here

Guppy compilation failed due to 1 previous error


In [9]:
from guppylang import guppy
from guppylang.std.builtins import owned
from guppylang.std.quantum import qubit, h, cx, discard, x

@guppy
def borrow_qubits(
    alice: qubit,
    bob: qubit
) -> None:
    x(alice)
    h(alice)
    cx(alice, bob)

borrow_qubits.check()

## Qubits Cannot be Leaked

Failed Example

In [12]:
from guppylang.std.quantum import qubit, h, measure

@guppy
def invalid() -> bool:
    alice = qubit()
    bob = qubit()
    h(alice)
    cx(alice, bob)
    return measure(alice)

invalid.check()

Error: Drop violation (at <In[12]>:6:4)
  | 
4 | def invalid() -> bool:
5 |     alice = qubit()
6 |     bob = qubit()
  |     ^^^ Variable `bob` with non-droppable type `qubit` is leaked

Help: Make sure that `bob` is consumed or returned to avoid the leak

Guppy compilation failed due to 1 previous error


In [13]:
from guppylang.std.quantum import qubit, h, measure, discard

@guppy
def invalid() -> bool:
    alice = qubit()
    bob = qubit()
    h(alice)
    cx(alice, bob)
    discard(bob)
    return measure(alice)

invalid.check()